### Debt Repayment Plan

#### Part 2 - Advanced Case

- Data
    - **N**  : Net monthly income
    - **M**  : Monthly required savings
    - **I<sub>j</sub>**  : Interest paid on loan to lender **j**
    

- Decision Variables
    - **S<sub>i</sub>**  : Monthly savings during period **i**
    - **P<sub>ij</sub>**  : Debt paid to lender **j** during month **i**
    - **B<sub>ij</sub>**  : Debt owed to lender **j** at the beginning of month **i**

In [1]:
using JuMP
import HiGHS

In [2]:
# Data
data = Dict(
    "loaner 1"        => Dict( "loan" => 3200, "term" => 4, "interest" => 3.8 / 100 ),
    "loaner 2"        => Dict( "loan" => 4700, "term" => 5, "interest" => 7.3 / 100 ),
    "loaner 3"        => Dict( "loan" => 6000, "term" => 7, "interest" => 4.8 / 100 ),
    "loaner 4"        => Dict( "loan" => 1500, "term" => 4, "interest" => 3.9 / 100 ),
    "loaner 5"        => Dict( "loan" => 2100, "term" => 4, "interest" => 8.1 / 100 ),
    "loaner 6"        => Dict( "loan" => 500 , "term" => 3, "interest" => 5.4 / 100 ),
    "loaner 7"        => Dict( "loan" => 100 , "term" => 1, "interest" => 3.2 / 100 ),
    "net_income"      => 4500,
    "minimum_monthly_savings" => 1000
);

In [3]:
max_term = maximum(data[loaner]["term"] for loaner in keys(data) if contains(loaner, "loaner"))
periods = 1:max_term

LENDERS = sort([loaner for loaner in keys(data) if contains(loaner, "loaner")])

7-element Vector{String}:
 "loaner 1"
 "loaner 2"
 "loaner 3"
 "loaner 4"
 "loaner 5"
 "loaner 6"
 "loaner 7"

In [4]:
function set_model_variables(
        model::Model, 
        periods::UnitRange, 
        NET_INCOME::Number, 
        MINIMUM_SAVINGS::Number,
        LENDERS::Array{String}
    )
    # Monthly savings
    Savings = @variable(model, Saving[periods] >= MINIMUM_SAVINGS );

    # Monthly debt reimbursed to each lender
    Payments = @variable(model, Payment[periods, LENDERS] >= 0);

    # Monthly balance due to each lender
    Balances = @variable(model, Balance[cat(periods, periods[end] + 1, dims = 1), LENDERS] >= 0);
    
    return Savings, Payments, Balances
end

set_model_variables (generic function with 1 method)

In [5]:
function set_model_constraints( 
        model::Model, 
        debt::Dict,
        NET_INCOME::Int,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balances::JuMP.Containers.DenseAxisArray
    )
    loan_label = "loan"; period_label = "term"; interest_label = "interest";

    # Debt owed at the beginning of period 1 to each lender
    @constraint( model, [lender in keys(debt)], Balances[1, lender] == debt[lender][loan_label] )

    # Ensure that all debt is paid off by the grace period
    @constraint( model, [lender in keys(debt)] , Balances[debt[lender][period_label] + 1, lender] == 0 )

    # Debt owed the beginning of period 2...end
    @constraint( model, 
        [i in periods, lender in keys(debt)], 
        Balances[i + 1, lender] == (1 + debt[lender][interest_label]) * (Balances[i, lender] - Payments[i, lender]) 
    )

    # Monthly expense can't exceed net income
    @constraint( model, [i in periods], Savings[i] + sum(Payments[i, LENDERS]) <= NET_INCOME )

end

set_model_constraints (generic function with 1 method)

#### Solve Model

In [6]:
model = Model( HiGHS.Optimizer )

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [7]:
Savings, Payments, Balance = set_model_variables(model, periods, data["net_income"], data["minimum_monthly_savings"], LENDERS)

(1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:7
And data, a 7-element Vector{VariableRef}:
 Saving[1]
 Saving[2]
 Saving[3]
 Saving[4]
 Saving[5]
 Saving[6]
 Saving[7], 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 1:7
    Dimension 2, ["loaner 1", "loaner 2", "loaner 3", "loaner 4", "loaner 5", "loaner 6", "loaner 7"]
And data, a 7×7 Matrix{VariableRef}:
 Payment[1,loaner 1]  Payment[1,loaner 2]  …  Payment[1,loaner 7]
 Payment[2,loaner 1]  Payment[2,loaner 2]     Payment[2,loaner 7]
 Payment[3,loaner 1]  Payment[3,loaner 2]     Payment[3,loaner 7]
 Payment[4,loaner 1]  Payment[4,loaner 2]     Payment[4,loaner 7]
 Payment[5,loaner 1]  Payment[5,loaner 2]     Payment[5,loaner 7]
 Payment[6,loaner 1]  Payment[6,loaner 2]  …  Payment[6,loaner 7]
 Payment[7,loaner 1]  Payment[7,loaner 2]     Payment[7,loaner 7], 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 

In [8]:
debt = Dict(lender => data[lender] for lender in LENDERS)
set_model_constraints( model, debt, data["net_income"], Savings, Payments, Balance )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:7
And data, a 7-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 Saving[1] + Payment[1,loaner 1] + Payment[1,loaner 2] + Payment[1,loaner 3] + Payment[1,loaner 4] + Payment[1,loaner 5] + Payment[1,loaner 6] + Payment[1,loaner 7] <= 4500.0
 Saving[2] + Payment[2,loaner 1] + Payment[2,loaner 2] + Payment[2,loaner 3] + Payment[2,loaner 4] + Payment[2,loaner 5] + Payment[2,loaner 6] + Payment[2,loaner 7] <= 4500.0
 Saving[3] + Payment[3,loaner 1] + Payment[3,loaner 2] + Payment[3,loaner 3] + Payment[3,loaner 4] + Payment[3,loaner 5] + Payment[3,loaner 6] + Payment[3,loaner 7] <= 4500.0
 Saving[4] + Payment[4,loaner 1] + Payment[4,loaner 2] + Payment[4

In [9]:
@objective(model, Max, sum(Savings))

Saving[1] + Saving[2] + Saving[3] + Saving[4] + Saving[5] + Saving[6] + Saving[7]

In [10]:
print(model)

In [11]:
function solve_infeasible(model)
    optimize!(model)
    if termination_status(model) == OPTIMAL
        return objective_value(model)
    else
        @warn("The model was not solved correctly.")
        return nothing
    end
end

solve_infeasible(model)

Presolving model
22 rows, 43 cols, 79 nonzeros
11 rows, 32 cols, 57 nonzeros
11 rows, 32 cols, 57 nonzeros
Presolve : Reductions: rows 11(-59); columns 32(-80); elements 57(-160)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.9999920692e+00 Ph1: 5(5); Du: 5(4.99999) 0s
         22    -1.1337912211e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 22
Objective value     :  1.1337912211e+04
HiGHS run time      :          0.00


11337.912211234663

In [12]:
using Printf
function print_payment_plan( 
        model::Model,
        debt::Dict,
        Savings::JuMP.Containers.DenseAxisArray, 
        Payments::JuMP.Containers.DenseAxisArray, 
        Balances::JuMP.Containers.DenseAxisArray
    )
    @printf("| %-10s | %-10s | %-15s | %-10s | %-10s | %-15s | %-10s | %-12s |\n",
        "Period", "Lender", "Initial Balance", "Payment", "Savings", 
        "Int. Rate (%)", "Interest", "End Balance"
    )
    for i in periods
        first_print = true
        for lender in LENDERS
            start_balance = value(Balances[i, lender])
            end_balance = value(Balances[i, lender]) - value(Payments[i, lender])
            payment = value(Payments[i, lender])
            interest = ( 
                i == 1 ? 0 
                : 
                value(Balances[i, lender]) - (value(Balances[i - 1, lender] ) - value(Payments[i - 1, lender])) 
            )
            @printf("| %10s | %-10s | %15.2f | %10.2f | %10s | %15.2f | %10.2f | %12.2f |\n", 
                if first_print i else "-" end,
                lender,
                start_balance > 0 ? start_balance : 0.00,
                payment > 0 ? payment : 0.00, 
                "-",
                debt[lender]["interest"],
                interest > 0 ? interest : 0.00,
                end_balance > 0 ? end_balance : 0.00
            )
            first_print = false
        end
        @printf("| %10s | %-10s | %15s | %10s | %10.2f | %15s | %10s | %12s |\n", 
                "-",
                "-",
                "-", 
                "-", 
                value(Savings[i]),
                "-",
                "-",
                "-"
            )
    end
end;

In [13]:
if termination_status(model) == OPTIMAL
    print_payment_plan(model, debt, Savings, Payments, Balance)
end

| Period     | Lender     | Initial Balance | Payment    | Savings    | Int. Rate (%)   | Interest   | End Balance  |
|          1 | loaner 1   |         3200.00 |       0.00 |          - |            0.04 |       0.00 |      3200.00 |
|          - | loaner 2   |         4700.00 |    1300.00 |          - |            0.07 |       0.00 |      3400.00 |
|          - | loaner 3   |         6000.00 |       0.00 |          - |            0.05 |       0.00 |      6000.00 |
|          - | loaner 4   |         1500.00 |       0.00 |          - |            0.04 |       0.00 |      1500.00 |
|          - | loaner 5   |         2100.00 |    2100.00 |          - |            0.08 |       0.00 |         0.00 |
|          - | loaner 6   |          500.00 |       0.00 |          - |            0.05 |       0.00 |       500.00 |
|          - | loaner 7   |          100.00 |     100.00 |          - |            0.03 |       0.00 |         0.00 |
|          - | -          |               - |          -

In [14]:
@printf("%119s", '\U2111')

                                                                                                                      ℑ